# Node Inputs

In [1]:
import json
from subprocess import call
import numpy as np
import math

# Important parameters

# Total number of nodes
n = [16,32,64,96,128,160,192,224,256]

# Number of witnesses
w = [int(min(4*np.log2(x),x)) for x in n]
v = [int(min(5*np.log2(x),x)) for x in n]
u = [x//2 + 1 for x in w]

# Number of Samples
s = [int(min(10*np.log2(x),x)) for x in n]
th = [(2*x)//3 + 1 for x in s]

In [2]:
n

[16, 32, 64, 96, 128, 160, 192, 224, 256]

In [3]:
v

[16, 25, 30, 32, 35, 36, 37, 39, 40]

In [4]:
s

[16, 32, 60, 65, 70, 73, 75, 78, 80]

In [2]:
# Brachas's inputs

for i in range(len(n)):
    f = math.ceil(n[i]/3) - 1
    experiment = {"protocol": "bracha"}
    experiment["parameters"] = {"n": n[i], "f": f}
    
    # Serializing json
    json_object = json.dumps(experiment, indent=4)

    # Writing to sample.json
    with open("Experiments/brachaInput" + str(i) + ".json", "w") as outfile:
        outfile.write(json_object)

In [3]:
# Reliable Accountability

# Fixed Parameters
wr = 10
vr = 10
recoveryTm = 60000000000
idSize = 256
numberBins = 16

for i in range(len(n)):
    f = math.ceil(n[i]/3) - 1
    experiment = {"protocol": "reliable_accountability"}
    experiment["parameters"] = {"n": n[i], "f": f,
                               "w": w[i], "v": v[i], "wr": 10, "vr": 10,
                               "u": u[i], "recovery_timeout": recoveryTm,
                               "node_id_size": idSize, "number_of_bins": numberBins}
    
    json_object = json.dumps(experiment, indent=4)

    with open("Experiments/witnessInput" + str(i) + ".json", "w") as outfile:
        outfile.write(json_object)

In [4]:
# Scalable Broadcast

for i in range(len(n)):
    experiment = {"protocol": "scalable"}
    experiment["parameters"] = {"n": n[i], "f": 0,
                               "g_size": s[i], "e_size": s[i], "r_size": s[i], "d_size": s[i],
                               "e_threshold": th[i], "r_threshold": th[i], "d_threshold": th[i],
                                "clean_up_timeout": 10000000000}
    
    json_object = json.dumps(experiment, indent=4)

    with open("Experiments/scalableInput" + str(i) + ".json", "w") as outfile:
        outfile.write(json_object)

# Simulation config files

### Changes bellow are mainly on the system's throughput
### Transactions / second
#targetThr = [int((2**(1/2))**(2+i)) for i in range(15)]
targetThr = [2*(i+1) for i in range(64)]

### Requests are evengly split among nodes according to the formula  

###    thr = n * thr_n, where thr_n is a node's requests per second

totalTime = 180000000000 # 180 seconds in nanoseconds
baseDelay = 1000000000 # 1 second in nanoseconds
for i in range(len(n)):
    for th in targetThr:
        # Fixed Parameters
        config = {"topology": "singleTopo.json",
                  "hostBand": 10, "hostQueue": 10000,
                  "hostDelay": "5ms", "hostLoss": 1}
        
        config["testNodes"] = {"inTestNodes": False, "outTestNodes": False,
                               "inTopoDelay": "25ms", "outTopoDelay": "25ms"}
        
        trDelay = (n[i]/th) * baseDelay
        totalTr = totalTime/trDelay
        
        config["simulationTime"] = int(totalTime/baseDelay) + 120
        config["numberTransactions"] = int(totalTr)
        config["transactionDelay"] = int(trDelay)
        
        json_object = json.dumps(config, indent=4)

        with open("Experiments/config" + str(i) + "_" + str(th) + ".json", "w") as outfile:
            outfile.write(json_object)

In [6]:
## Update ##
# Requests are now spread throughout 16 processes only

#    thr = 16 * thr_n, where thr_n is a node's requests per second

targetThr = [math.ceil(2**((i+6)/3)) for i in range(22)]

totalTime = 150000000000 # 150 seconds in nanoseconds
baseDelay = 1000000000 # 1 second in nanoseconds

for th in targetThr:
    # Fixed Parameters
    config = {"topology": "singleTopo.json",
              "hostBand": 100, "hostQueue": 10000000000,
              "hostDelay": "5ms", "hostLoss": 0}

    config["testNodes"] = {"inTestNodes": False, "outTestNodes": False,
                           "inTopoDelay": "25ms", "outTopoDelay": "25ms"}

    trDelay = (16/th) * baseDelay
    totalTr = totalTime/trDelay

    config["simulationTime"] = int(totalTime/baseDelay) + 150
    config["numberTransactions"] = int(totalTr)
    config["transactionDelay"] = int(trDelay)

    json_object = json.dumps(config, indent=4)

    with open("Experiments/config" + "16" + "_" + str(th) + ".json", "w") as outfile:
        outfile.write(json_object)

# Stress Test

In [9]:
# Having numerous tests with the same configuration

for i in range(5):
    
        # Fixed Parameters
        config = {"topology": "singleTopo.json",
                  "hostBand": 10, "hostQueue": 10000,
                  "hostDelay": "5ms", "hostLoss": 1}

        config["testNodes"] = {"inTestNodes": False, "outTestNodes": False,
                               "inTopoDelay": "25ms", "outTopoDelay": "25ms"}


        config["simulationTime"] = 250
        config["numberTransactions"] = -1
        config["transactionDelay"] = 180000000000

        json_object = json.dumps(config, indent=4)

        with open("Experiments/stress" + str(i) + ".json", "w") as outfile:
            outfile.write(json_object)

# Best case test

In [8]:
for i in range(5):
    # Fixed Parameters
        config = {"topology": "singleTopo.json",
                  "hostBand": 100, "hostQueue": 10000,
                  "hostDelay": "100ms", "hostLoss": 1}

        config["testNodes"] = {"inTestNodes": False, "outTestNodes": False,
                               "inTopoDelay": "25ms", "outTopoDelay": "25ms"}


        config["simulationTime"] = 250
        config["numberTransactions"] = -2
        config["transactionDelay"] = 180000000000

        json_object = json.dumps(config, indent=4)

        with open("Experiments/best" + str(i) + ".json", "w") as outfile:
            outfile.write(json_object)